In [ ]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm

In [ ]:
data_path = 'data/'
subm_path = data_path + 'subm/'

train_file = data_path + 'ru_train.csv'
test_file = data_path +'ru_test.csv'

In [ ]:
train_df = pd.read_csv(train_file)
print(train_df.shape)
train_df.head()

In [ ]:
train_df.info()

In [ ]:
diff = data[data.before != data.after]
print(diff.shape)
diff.head(10)

## Analyze classes:

In [ ]:
classes = train_df['class'].unique(); classes

In [ ]:
train_df[train_df['class'] == 'PUNCT']['before'].unique()

In [ ]:
def class_sizes(df):
    class_sizes = {}
    df_size = df.shape[0]
    for cl in tqdm(classes):
        #write_file(INPUT_PATH, train_file[:-4] + '.' + cl + '.csv', train_df[train_df['class'] == cl])
        cl_size = df[df['class'] == cl].shape[0]
        cl_unique_size =  df[df['class'] == cl]['before'].unique().size
        class_sizes[cl] = (int((cl_size  / df_size) * 100), cl_size,cl_unique_size)
    return class_sizes

In [ ]:
class_sizes_of_train = class_sizes(train_df); class_sizes_of_train

In [ ]:
class_sizes_of_diff = class_sizes(diff); class_sizes_of_diff

Worth to note the 'PUNCT': (0, 0, 0) for the differential dataframe. That means we could hard code all PUNCT elements as 1->1 translation. :)
'VERBATIM': (0, 12944, 73) - the variance in whole set is very small. Possible could be also translated with hard code.

### Create a separate file for each class and manually analyze all of them.

In [ ]:
[train_df[train_df['class'] == c][['before','after']].to_csv(train_file[:-3] + c + '.csv', encoding='UTF-8', index=False) for c in classes]

### The results of analysis

| class    |=before?|spaces? | punctuation? | letters? | digits? | size: all/unique  |  samples
|:---------|:------:|:-------:|:-----------:|:--------:|:-------:|:-------------|:-------
| PLAIN    |    -+         |  -      |  -+              | +        |    -  |  7360439/696262 |  Корневище,Корневище и,и  —,до  An,а_trans н_trans  je,д_trans ж_trans е_trans
| DECIMAL   |    -         |  +      | ,-,           |  +         |  +    | 7,299/3726  |    "3,04" '220 тыс.'
| DATE      |   -         |   +      | .            |   +         |  +  | 185,961/37,215 | '10 марта 2015' '21.12.01' '4 ноября 2004 г.'
| PUNCT     |   +         |   -       |+             |  -         |  -  | 2288640/22  |  ['.', ',', '(', ')', '—', '?', ':', '«', '»', '/', '"', '[', ';', ']', "'", '!', '--', '¿', '---', pipe, '-----', '¡']
| ORDINAL   |   -+        |   -      | +            |  +         |  +  | 46738/3995 |  9-й,девятый  XIII,тринадцатого  —1918,тысяча девятьсот восемнадцатого  3й,третий
| VERBATIM  |   -+        |   -      | -            |   + non-eng | + sub/super | 157,914/2,505 | '-', '兄', '貴'
| LETTERS   |   -        |    -+     | +            |   +        |   -  |  189528/16148  |   М.,м Э. Х.,э х ISBN,i s b n IV,i v NZ,n z
| CARDINAL  |   -+       |    -+     | -            |   -        |   +  |   272442/10555 |  2,два 1995,тысяча девятьсот девяносто пять  12 500,двенадцать тысяч пятьсот  -73,минус семь три 6-ти,шести
| MEASURE   |   -+       |    +      | +            |   +        |   +  |     40534/15395 | 2 Р.,2 Р.  600с.,шестьсот секунд  90 %,девяносто процентов 3 см,3 см  "2,41 км².",две целых и сорок одну сотую квадратного километра
| TELEPHONE |   -        |    +      |  +           |   -        |   +  |     10088, 8496 |  2 (1997)15,два sil девятнадцать девяносто семь sil пятнадцать  13 (1) 2013,тринадцать sil один sil двадцать тринадцать 04061,ноль сорок шестьдесят один
| ELECTRONIC |  -+      |     -     |  +            |   +        |   + |  5832/3710 | Корреспондент.net,к о р р е с п о н д е н т точка н_trans е_trans т_trans ::,::  .xls,точка x l s Popeater.com,п_trans о_trans п_trans и_trans т_trans е_trans р_trans точка к_trans о_trans м_trans
| DECIMAL   |   -+     |       +    |   +           |    +       |    + |  7297/3726 | "107,2",сто семь целых и две десятых 180 млн.,сто восемьдесят миллионов  100 тыс.,ста тысяч  "-49,50",минус сорок девять целых и пятьдесят сотых
| DIGIT     |   -       |     -     |  -            |   -        |   +  | 2012/294 | 08,ноль восемь !!! all started with '0'
| FRACTION  |   -       |     -+    |   -           |    -       |    + |      2460/954 | 2/3,две третьих  -717/714,-717/714  3½,три целых и одну вторую  1 / 20,одна двадцатая
| MONEY     |   -+      |     +      | -+           |   +        |   +  |     2690/2253 |  €5,€  "2,7 млрд евро",две целых и семь десятых миллиарда евро  22 151 USD,двадцати двух тысяч ста пятидесяти одного доллара сэ ш а   300 руб.,трехсот рублей
| TIME      |   -+       |     -+   |  +            | -+          |  +   |    1945, 762   |      ,23:25,двадцать три часа двадцать пять минут  06:53:05,06:53:05  11: 16,одиннадцать часов шестнадцать минут  AM2,два часа ночи

### What we can propose from looking to this analysis?

TODO